# Introduction à la librairie PyTorch -- Solutions
Matériel de cours rédigé par Pascal Germain, 2019
************

### Partie 1

In [ ]:
class regression_logistique:
    def __init__(self, rho=.01, eta=0.4, nb_iter=50, seed=None):
        self.rho = rho
        self.eta = eta
        self.nb_iter = nb_iter
        self.seed = seed
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32) 

        n, d = x.shape
        self.w = torch.randn(d, requires_grad=True)
        
        self.w_list = list() # Servira à garder une trace de la descente de gradient
        self.obj_list = list()
    
        for t in range(self.nb_iter+1):
            xw = x @ self.w
            w2 = self.w @ self.w
            loss = torch.mean(- y * xw + torch.log(1+torch.exp(xw))) + self.rho * w2 / 2
            
            self.w_list.append(np.array(self.w.detach()))
            self.obj_list.append(loss.item()) 
            if t == self.nb_iter: break 
        
            with torch.no_grad():
                loss.backward()
                self.w -= self.eta * self.w.grad
                
            self.w.grad.zero_()
                
    def prediction(self, x):
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32)
            pred = x @ self.w.detach()
        
        return np.array(pred.numpy() > .5, dtype=np.int)

In [1]:
class regression_logistique_avec_biais:
    def __init__(self, rho=.01, eta=0.4, nb_iter=50, seed=None):
        self.rho = rho
        self.eta = eta
        self.nb_iter = nb_iter
        self.seed = seed
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32) 

        n, d = x.shape
        self.w = torch.randn(d, requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)
        
        self.w_list = list() # Servira à garder une trace de la descente de gradient
        self.obj_list = list()
    
        for t in range(self.nb_iter+1):
            xw = x @ self.w + self.b
            w2 = self.w @ self.w
            loss = torch.mean(- y * xw + torch.log(1+torch.exp(xw))) + self.rho * w2 / 2
            
            self.w_list.append(np.array(self.w.detach()))
            self.obj_list.append(loss.item()) 
            if t == self.nb_iter: break 
        
            with torch.no_grad():
                loss.backward()
                self.w -= self.eta * self.w.grad
                self.b -= self.eta * self.b.grad
                
            self.w.grad.zero_()
            self.b.grad.zero_()            
                
    def prediction(self, x):
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32)
            pred = x @ self.w.detach() + self.b.item()
            
        return np.array(pred.numpy() > .5, dtype=np.int)

### Partie 2

In [2]:
class reseau_classification:
    def __init__(self, nb_neurones=4, eta=0.4, alpha=0.1, nb_iter=50, seed=None):
        self.nb_neurones = nb_neurones
        self.eta = eta
        self.alpha = alpha
        self.nb_iter = nb_iter
        self.seed = seed
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

        n, d = x.shape
        
        self.model = nn.Sequential(
            torch.nn.Linear(d, self.nb_neurones),
            torch.nn.ReLU(),
            torch.nn.Linear(self.nb_neurones, 1),
            torch.nn.Sigmoid()
        )
        
        perte_logistique = nn.BCELoss()
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.eta, momentum=self.alpha)
        
        self.obj_list = list()
    
        for t in range(self.nb_iter+1):
            y_pred = self.model(x)
            
            loss = perte_logistique(y_pred, y)
                  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            self.obj_list.append(loss.item())
                
    def prediction(self, x):
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32)
            pred = self.model(x)
            
        return np.array(pred.detach() > .5, dtype=np.int)[:,0]